# How to speed up a pandas query 10x with Dask

This notebook requires you to download a dataset from S3 to your local machine.

This notebook works with the `crt-004` environment.  Run `conda env create -f envs/crt-004.yml` to create this environment.

Here is the [full blog post](https://coiled.io/blog/speed-up-pandas-query-10x-with-dask/) that describes the computations in this notebook.

## Localhost data setup

You need to download some data from S3 to run the examples in this notebook.  You need to either setup the AWS CLI on your machine and run the following commands or download the data using another approach.

In [1]:
!mkdir data

In [2]:
!aws s3 cp s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_single.csv data/

download: s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_single.csv to data/N_1e8_K_1e2_single.csv


The N_1e8_K_1e2_single.csv file contains XX rows and is 4.8 GB.  It's not a huge amount of data, but large enough for Dask to provide meaningful performance improvements compared to pandas.

## Baseline: Just use Pandas

In [25]:
import pandas as pd

In [26]:
%%time
df = pd.read_csv("data/N_1e8_K_1e2_single.csv")
df.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"})

CPU times: user 58.9 s, sys: 32.8 s, total: 1min 31s
Wall time: 3min 38s


,v1
id1,
id001,1999081
id002,2002870
id003,1998389
id004,2001091
id005,1999770
...,...
id096,2001015
id097,1995014
id098,2000933


In [27]:
df.dtypes

id1     object
id2     object
id3     object
id4      int64
id5      int64
id6      int64
v1       int64
v2       int64
v3     float64
dtype: object

## Optimization #1: Use Dask & inefficient dtypes

In [3]:
import dask.dataframe as dd

In [4]:
dd.read_csv("data/N_1e8_K_1e2_single.csv").dtypes

id1     object
id2     object
id3     object
id4      int64
id5      int64
id6      int64
v1       int64
v2       int64
v3     float64
dtype: object

In [5]:
dtypes = {
    "id1": "object",
    "id2": "object",
    "id3": "object",
    "id4": "object",
    "id5": "object",
    "id6": "object",
    "v1": "object",
    "v2": "object",
    "v3": "object",
}

In [6]:
ddf = dd.read_csv("data/N_1e8_K_1e2_single.csv", dtype=dtypes)

In [7]:
ddf.head()

,id1,id2,id3,id4,id5,id6,v1,v2,v3
0,id048,id035,id0000608844,22,63,812586,3,3,95.569069
1,id080,id099,id0000466449,40,79,682109,2,13,97.012438
2,id035,id041,id0000573987,81,10,800483,0,1,74.776325
3,id086,id047,id0000776204,53,72,212890,3,14,45.157498
4,id009,id091,id0000608718,93,65,655745,2,0,51.132162


In [8]:
ddf.dtypes

id1    object
id2    object
id3    object
id4    object
id5    object
id6    object
v1     object
v2     object
v3     object
dtype: object

In [9]:
ddf["v1"] = ddf["v1"].astype("int64")

In [10]:
%%time
ddf.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 1min 50s, sys: 28 s, total: 2min 18s
Wall time: 2min 17s


,v1
id1,
id001,1999081
id002,2002870
id003,1998389
id004,2001091
id005,1999770
...,...
id096,2001015
id097,1995014
id098,2000933


### Optimization #2: Use better dtypes

In [11]:
better_dtypes = {
    "id1": "string[pyarrow]",
    "id2": "string[pyarrow]",
    "id3": "string[pyarrow]",
    "id4": "int64",
    "id5": "int64",
    "id6": "int64",
    "v1": "int64",
    "v2": "int64",
    "v3": "float64",
}

In [12]:
ddf = dd.read_csv("data/N_1e8_K_1e2_single.csv", dtype=better_dtypes)

In [13]:
%%time
ddf.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 1min 28s, sys: 22.4 s, total: 1min 50s
Wall time: 1min 24s


,v1
id1,
id001,1999081
id002,2002870
id003,1998389
id004,2001091
id005,1999770
...,...
id096,2001015
id097,1995014
id098,2000933


## Optimization #3: Split dataset into many CSV files

In [14]:
ddf.repartition(partition_size="100MB").to_csv("data/csvs")

['/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/000.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/001.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/002.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/003.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/004.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/005.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/006.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/007.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/008.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/009.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/010.part',
 '/Users/powers/Documents/code/coiled/coiled-resources/blogs/data/csvs/011.part',
 '/Users/powers/

In [15]:
ddf = dd.read_csv("data/csvs/*", dtype=better_dtypes)

In [16]:
%%time
ddf.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 1min 36s, sys: 10.8 s, total: 1min 46s
Wall time: 1min 4s


,v1
id1,
id001,1999081
id002,2002870
id003,1998389
id004,2001091
id005,1999770
...,...
id096,2001015
id097,1995014
id098,2000933


## Optimization #4: Split dataset into multiple uncompressed Parquet files

In [17]:
ddf.to_parquet("data/parquet", engine="pyarrow", compression=None)

(None,)

In [18]:
ddf = dd.read_parquet("data/parquet", engine="pyarrow")

In [19]:
%%time
ddf.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 40.7 s, sys: 13.3 s, total: 54 s
Wall time: 41.3 s


,v1
id1,
id001,1999081
id002,2002870
id003,1998389
id004,2001091
id005,1999770
...,...
id096,2001015
id097,1995014
id098,2000933


## Optimization #5: Using Snappy compressed Parquet

In [20]:
ddf.to_parquet("data/snappy-parquet", engine="pyarrow", compression="snappy")

(None,)

In [21]:
ddf = dd.read_parquet("data/snappy-parquet", engine="pyarrow")

In [22]:
%%time
ddf.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 44.8 s, sys: 14.8 s, total: 59.6 s
Wall time: 42.4 s


,v1
id1,
id001,1999081
id002,2002870
id003,1998389
id004,2001091
id005,1999770
...,...
id096,2001015
id097,1995014
id098,2000933


## Optimization #6: Leverage Parquet column pruning

In [23]:
ddf = dd.read_parquet("data/snappy-parquet", engine="pyarrow", columns=["id1", "v1"])

In [24]:
%%time
ddf.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 21.4 s, sys: 958 ms, total: 22.3 s
Wall time: 16.8 s


,v1
id1,
id001,1999081
id002,2002870
id003,1998389
id004,2001091
id005,1999770
...,...
id096,2001015
id097,1995014
id098,2000933


## Cleanup (optional)

You don't need to keep the CSV data file you downloaded for this example on your machine anymore if you're not going to use it again.  I personally love having this data on my machine for experimentation purposes, but you can delete it if you don't need it anymore.

In [ ]:
!rm -rf data